# Paper 1: Memory as Baseline Deviation

**DOI:** [10.5281/zenodo.14538419](https://doi.org/10.5281/zenodo.14538419)

This notebook walks through the core equation of the MBD framework:

$$B(t+1) = B(t)(1 - \lambda) + I(t) \cdot \lambda$$

where:
- $B(t)$ is the **baseline** (the agent's resting personality state) at time $t$
- $I(t)$ is the **input signal** (trauma, joy, novelty) at time $t$
- $\lambda \in [0, 1]$ is the **plasticity parameter** (how deeply the event rewrites the baseline)

Every plot below is generated from the actual framework source code.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath('..'))

import numpy as np
import matplotlib.pyplot as plt

from analysis.trauma_model import (
    Baseline, TraumaForm, update_baseline, update_kappa, Interaction
)

print('Framework loaded.')

Framework loaded.


## 1. The Baseline Vector

An agent's personality is not a label — it is a **vector** in state space.
We use three dimensions from the VAD (Valence–Activation–Dominance) model:

| Dimension | Range | Meaning |
|-----------|-------|---------|
| **Valence** | -1 to +1 | Negative ← → Positive affect |
| **Activation** | -1 to +1 | Calm ← → Energised |
| **Dominance** | -1 to +1 | Submissive ← → Dominant |

The baseline is where you *return to* after perturbation — your emotional
centre of gravity.

In [ ]:
B0 = Baseline([0.5, -0.2, 0.1])
print(f'Initial baseline B0 = {B0.vector}')
print(f'  Valence:    {B0.vector[0]:+.2f}  (mildly positive)')
print(f'  Activation: {B0.vector[1]:+.2f}  (slightly calm)')
print(f'  Dominance:  {B0.vector[2]:+.2f}  (near neutral)')

## 2. Trauma as a Mathematical Object

A `TraumaForm` packages three things:
- **input_signal**: A vector that pushes the baseline in some direction
- **lambda_learning_rate** ($\lambda$): How deeply this event rewrites the baseline
- **description**: Human-readable label

High $\lambda$ = the event *changes who you are*. Low $\lambda$ = a ripple that fades.

In [2]:
events = [
    TraumaForm(np.array([-0.8,  0.5, -0.6]), 0.3, 'Sudden loss'),
    TraumaForm(np.array([-0.3,  0.2, -0.2]), 0.1, 'Chronic stress'),
    TraumaForm(np.array([ 0.4, -0.1,  0.3]), 0.2, 'New friendship'),
    TraumaForm(np.array([-0.9,  0.7, -0.8]), 0.5, 'Betrayal'),
    TraumaForm(np.array([ 0.6,  0.3,  0.4]), 0.15,'Milestone achievement'),
    TraumaForm(np.array([ 0.7, -0.2,  0.5]), 0.4, 'Deep recovery'),
]

hdr = ('Event', 'Signal', 'lambda')
print(f'{hdr[0]:<25} {hdr[1]:>25} {hdr[2]:>7}')
print('-' * 60)
for e in events:
    sig_str = np.array2string(e.input_signal, precision=1, separator=', ')
    print(f'{e.description:<25} {sig_str:>25} {e.lambda_learning_rate:7.2f}')

Event                                        Signal  lambda
------------------------------------------------------------
Sudden loss                      [-0.8,  0.5, -0.6]    0.30
Chronic stress                   [-0.3,  0.2, -0.2]    0.10
New friendship                   [ 0.4, -0.1,  0.3]    0.20
Betrayal                         [-0.9,  0.7, -0.8]    0.50
Milestone achievement               [0.6, 0.3, 0.4]    0.15
Deep recovery                    [ 0.7, -0.2,  0.5]    0.40


## 3. Baseline Evolution

We apply each event sequentially using the core equation:

$$B(t+1) = B(t)(1 - \lambda) + I(t) \cdot \lambda$$

Watch how the baseline drifts — some events barely move it (low $\lambda$),
while others permanently reshape the agent's resting state.

In [ ]:
B = Baseline(B0.vector.copy())
history = [B.vector.copy()]
labels = ['Initial']

for event in events:
    B = update_baseline(B, event)
    history.append(B.vector.copy())
    labels.append(event.description)

history = np.array(history)

hdr = ('Step', 'Event', 'Valence', 'Activation', 'Dominance')
print(f'{hdr[0]:<3} {hdr[1]:<25} {hdr[2]:>8} {hdr[3]:>11} {hdr[4]:>10}')
print('-' * 60)
for i, (lbl, row) in enumerate(zip(labels, history)):
    print(f'{i:<3} {lbl:<25} {row[0]:+8.4f} {row[1]:+11.4f} {row[2]:+10.4f}')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
x = range(len(history))
ax.plot(x, history[:, 0], 'o-', label='Valence', color='#3b82f6', linewidth=2.5)
ax.plot(x, history[:, 1], 's-', label='Activation', color='#ef4444', linewidth=2.5)
ax.plot(x, history[:, 2], '^-', label='Dominance', color='#10b981', linewidth=2.5)

ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=35, ha='right', fontsize=9)
ax.set_ylabel('State Value', fontsize=12)
ax.set_title('Baseline Trajectory Through Life Events', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='#475569', linestyle='--', alpha=0.5)

ax.set_facecolor('#0f172a')
fig.patch.set_facecolor('#0f172a')
ax.tick_params(colors='#94a3b8')
ax.xaxis.label.set_color('#94a3b8')
ax.yaxis.label.set_color('#94a3b8')
ax.title.set_color('#e2e8f0')
for spine in ax.spines.values():
    spine.set_color('#334155')
plt.tight_layout()
plt.show()

## 4. Plasticity Parameter Sweep

What happens when the *same event* hits at different plasticity levels?

Low $\lambda$ → the event barely registers (emotional callus, dissociation).  
High $\lambda$ → the event rewrites you completely (no buffer, no resistance).

In [ ]:
test_signal = np.array([-0.8, 0.5, -0.6])  # Sudden loss
lambdas = [0.01, 0.1, 0.3, 0.5, 0.8, 0.95]

fig, axes = plt.subplots(2, 3, figsize=(14, 8), sharey=True)
dim_labels = ['Valence', 'Activation', 'Dominance']
dim_colors = ['#3b82f6', '#ef4444', '#10b981']

for idx, lam in enumerate(lambdas):
    ax = axes[idx // 3][idx % 3]
    B_sweep = Baseline(B0.vector.copy())
    sweep_hist = [B_sweep.vector.copy()]
    for _ in range(5):
        trauma = TraumaForm(test_signal, lam, 'Same event')
        B_sweep = update_baseline(B_sweep, trauma)
        sweep_hist.append(B_sweep.vector.copy())
    sweep_hist = np.array(sweep_hist)
    for d in range(3):
        ax.plot(sweep_hist[:, d], 'o-', color=dim_colors[d],
                label=dim_labels[d] if idx == 0 else None, linewidth=2)
    ax.set_title(f'$\\lambda$ = {lam}', fontsize=12, color='#e2e8f0')
    ax.set_ylim(-1.1, 1.1)
    ax.grid(True, alpha=0.3)
    ax.set_facecolor('#0f172a')
    ax.tick_params(colors='#94a3b8')
    for spine in ax.spines.values():
        spine.set_color('#334155')

axes[0][0].legend(fontsize=9)
fig.suptitle('Same Event, Different Plasticity', fontsize=14, color='#e2e8f0')
fig.patch.set_facecolor('#0f172a')
plt.tight_layout()
plt.show()

## 5. Coupling Dynamics ($\kappa$)

Relationships between agents evolve through a coupling coefficient $\kappa$:

$$\frac{d\kappa}{dt} = \alpha(1 - N) - \beta\kappa$$

where:
- $\alpha$ controls how quickly coupling **grows** in low-novelty (familiar) situations
- $\beta$ controls natural coupling **decay**
- $N$ is the **novelty** of an interaction (high novelty = less coupling gain)

Below we trace a 10-interaction relationship arc — from strangers to bonded pair.

In [ ]:
interactions = [
    Interaction(0.9, 1.0, 'First meeting'),
    Interaction(0.7, 1.0, 'Second encounter'),
    Interaction(0.5, 2.0, 'Shared project'),
    Interaction(0.3, 1.5, 'Growing comfort'),
    Interaction(0.2, 2.0, 'Deep conversation'),
    Interaction(0.1, 3.0, 'Routine together'),
    Interaction(0.8, 1.0, 'Conflict / surprise'),
    Interaction(0.4, 2.0, 'Reconciliation'),
    Interaction(0.1, 4.0, 'Quiet coexistence'),
    Interaction(0.05, 5.0, 'Long-term bond'),
]

alpha = 0.2
beta = 0.05
kappa = 0.1
kappa_history = [kappa]
int_labels = ['Start']

for inter in interactions:
    kappa = update_kappa(kappa, inter, alpha, beta)
    kappa_history.append(kappa)
    int_labels.append(inter.description)

fig, ax = plt.subplots(figsize=(12, 5))
x = range(len(kappa_history))
ax.plot(x, kappa_history, 'o-', color='#f59e0b', linewidth=2.5, markersize=8)
ax.fill_between(x, kappa_history, alpha=0.2, color='#f59e0b')
ax.set_xticks(x)
ax.set_xticklabels(int_labels, rotation=35, ha='right', fontsize=9)
ax.set_ylabel('Coupling Strength ($\\kappa$)', fontsize=12)
ax.set_title('Relationship Arc: Coupling Over Time', fontsize=14)
ax.grid(True, alpha=0.3)

ax.set_facecolor('#0f172a')
fig.patch.set_facecolor('#0f172a')
ax.tick_params(colors='#94a3b8')
ax.xaxis.label.set_color('#94a3b8')
ax.yaxis.label.set_color('#94a3b8')
ax.title.set_color('#e2e8f0')
for spine in ax.spines.values():
    spine.set_color('#334155')
plt.tight_layout()
plt.show()

## 6. State-Space Trajectory

The baseline moves through 3D state space. Below are three 2D projection
planes, showing how the agent's resting personality drifts through
the life events defined above.

In [ ]:
pairs = [(0, 1, 'Valence', 'Activation'),
         (0, 2, 'Valence', 'Dominance'),
         (1, 2, 'Activation', 'Dominance')]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, (xi, yi, xl, yl) in zip(axes, pairs):
    ax.plot(history[:, xi], history[:, yi], 'o-', color='#8b5cf6',
            linewidth=2, markersize=8)
    ax.plot(history[0, xi], history[0, yi], 's', color='#10b981',
            markersize=12, label='Start', zorder=5)
    ax.plot(history[-1, xi], history[-1, yi], 'D', color='#ef4444',
            markersize=12, label='End', zorder=5)
    for i, lbl in enumerate(labels):
        ax.annotate(str(i), (history[i, xi], history[i, yi]),
                    fontsize=8, ha='center', va='bottom', color='#94a3b8')
    ax.set_xlabel(xl, fontsize=11)
    ax.set_ylabel(yl, fontsize=11)
    ax.legend(fontsize=9)
    ax.grid(True, alpha=0.3)
    ax.set_facecolor('#0f172a')
    ax.tick_params(colors='#94a3b8')
    ax.xaxis.label.set_color('#94a3b8')
    ax.yaxis.label.set_color('#94a3b8')
    for spine in ax.spines.values():
        spine.set_color('#334155')

fig.suptitle('Baseline Trajectory — Three Projection Planes', fontsize=14, color='#e2e8f0')
fig.patch.set_facecolor('#0f172a')
plt.tight_layout()
plt.show()

## Key Takeaways

1. **Memory is not storage — it is drift.** The baseline equation shows that
   every experience permanently alters the agent's resting state, weighted by
   the plasticity parameter $\lambda$.

2. **Plasticity is the vulnerability dial.** Low $\lambda$ = resilient but rigid.
   High $\lambda$ = adaptive but fragile. There is no "correct" setting — only
   trade-offs.

3. **Coupling grows in familiarity, fractures in novelty.** The $\kappa$ equation
   captures how relationships strengthen through routine and weaken through
   disruption — but conflict followed by reconciliation can ultimately strengthen
   the bond.

4. **State-space trajectories are fingerprints.** Two agents who experience the
   same events at different plasticity levels will trace completely different
   paths through personality space.

---

*Everett, B. (2024). Memory as Baseline Deviation (MBD): A Formal Framework
for Encoding Persistent Trauma in Autonomous Agents. Zenodo.*  
[10.5281/zenodo.14538419](https://doi.org/10.5281/zenodo.14538419)